In [2]:
!pip install numpy pandas

     |████████████████████████████████| 14.6 MB 305 kB/s eta 0:00:01
     |████████████████████████████████| 10.1 MB 122 kB/s eta 0:00:01
     |████████████████████████████████| 510 kB 371 kB/s eta 0:00:01
You should consider upgrading via the '/home/nghia/Documents/SWHGD/.env/bin/python -m pip install --upgrade pip' command.


In [26]:
import numpy as np
import pandas as pd
from datetime import datetime
import datetime as dt

In [9]:
# df = pd.read_csv("/home/sv/data/snapshot_fork_duplicates_removed_fixed.csv.gz", header=0, compression="gzip", names=["snapshot_id", "fork_id", "date"])
df = pd.read_csv("snapshot_fork_duplicates_removed_fixed.csv.gz", header=0, compression="gzip", names=["snapshot_id", "fork_id", "date"])
df.head()

,snapshot_id,fork_id,date
0,10000129,10000126,1433557281
1,1000048,1000047,1410017561
2,10002669,10002668,1389857537
3,10002835,42351765,1418498280
4,10002835,25311588,1398130898


In [17]:
df['year'] = df.apply(lambda row: datetime.utcfromtimestamp(row['date']).year, axis=1)
df['month'] = df.apply(lambda row: datetime.utcfromtimestamp(row['date']).month, axis=1)
df['day'] = df.apply(lambda row: datetime.utcfromtimestamp(row['date']).day, axis=1)
df['week'] = df.apply(lambda row: datetime.utcfromtimestamp(row['date']).isocalendar()[1], axis=1)

In [27]:
dt.date(2020, 12, 31).isocalendar()[1]

53

In [18]:
df.head()

,snapshot_id,fork_id,date,year,month,day,week
0,10000129,10000126,1433557281,2015,6,6,23
1,1000048,1000047,1410017561,2014,9,6,36
2,10002669,10002668,1389857537,2014,1,16,3
3,10002835,42351765,1418498280,2014,12,13,50
4,10002835,25311588,1398130898,2014,4,22,17


In [12]:
df['year'].nunique()

9

In [20]:
snapshot_fork_dict = dict()
for i, row in df.iterrows():
    if row.snapshot_id not in snapshot_fork_dict:
        snapshot_fork_dict[row.snapshot_id] = dict()

    if row.year not in snapshot_fork_dict[row.snapshot_id]:
        # count of year, dictitionary for month, dictionary for week number
        snapshot_fork_dict[row.snapshot_id][row.year] = [1, dict(), dict()] 
    else:
        snapshot_fork_dict[row.snapshot_id][row.year][0] += 1
    
    if row.month not in snapshot_fork_dict[row.snapshot_id][row.year][1]:
        snapshot_fork_dict[row.snapshot_id][row.year][1][row.month] = 1
    else:
        snapshot_fork_dict[row.snapshot_id][row.year][1][row.month] += 1
    
    if row.week not in snapshot_fork_dict[row.snapshot_id][row.year][2]:
        snapshot_fork_dict[row.snapshot_id][row.year][2][row.week] = 1
    else:
        snapshot_fork_dict[row.snapshot_id][row.year][2][row.week] += 1

print("The number of project with forks:", len(snapshot_fork_dict))
for snapshot, revision_date_set in snapshot_fork_dict.items():
    print(snapshot, len(revision_date_set))

The number of project with forks: 16
10000129 1
1000048 1
10002669 1
10002835 6
1000355 1
10004260 1
10004589 3
1000482 4
10005216 2
10006621 1
10008940 2
10010646 1
10010708 2
10010985 1
10011144 1
10011244 9


In [30]:
def count_time_units_exists(months, unit_per_year=12):
    units_exists = 0
    first_year, first_month = months[0]
    last_year, last_month = months[-1]
    if last_year == first_year:
        units_exists = last_month - first_month + 1
    elif last_month < first_month:
        extra_months = unit_per_year * (last_year - first_year) - 1
        units_exists = last_month + (12 - first_month + 1) + extra_months
    else:
        extra_months = unit_per_year * (last_year - first_year)
        units_exists = last_month - first_month + extra_months
    return units_exists

def is_steady(units, unit_per_year=12):
    year, unit = units[-1]
    num_iterations = unit_per_year
    while num_iterations:
        if (year, unit) not in units:
            return False
        unit -= 1
        if unit == 0:
            unit = unit_per_year
            year -= 1
        num_iterations -= 1
    return True

snapshot_ids = []
count_forks = []
count_years_exist = [] # the number of years between the first and last forks
count_months_exist = []
count_weeks_exist = []
mean_forks_per_year = []
mean_forks_per_month = []
mean_forks_per_week = []
count_years_with_forks = []
count_months_with_forks = []
count_week_with_forks = []
steady_month = []
steady_week = []
for snapshot, revision_date_set in snapshot_fork_dict.items():
    forks_year = []
    forks_month = []
    forks_week = []
    for year, (frequency, revision_date_set_month, revision_date_set_week) in revision_date_set.items():
        forks_year.append((year, frequency))
        for month, freq in revision_date_set_month.items():
            forks_month.append((year, month, freq))
        for week, freq in revision_date_set_week.items():
            forks_week.append((year, week, freq))
    forks_year.sort()
    forks_month.sort()
    forks_week.sort()
    count_fork = 0
    count_years_with_fork = 0
    count_months_with_fork = 0
    count_weeks_with_fork = 0
    years = []
    months = []
    weeks = []
    for year, frequency in forks_year:
        years.append(year)
        count_fork += frequency
        count_years_with_fork += 1
    for year, month, freq in forks_month:
        months.append((year, month))
        count_months_with_fork += 1
    for year, week, freq in forks_week:
        weeks.append((year, week))
        count_weeks_with_fork += 1
    
    months_exist = count_time_units_exists(months, 12)
    weeks_exist = count_time_units_exists(weeks, 53)
        
    snapshot_ids.append(snapshot)
    count_forks.append(count_fork)
    count_years_with_forks.append(count_years_with_fork)
    count_months_with_forks.append(count_months_with_fork)
    years_exist = years[-1] - years[0] + 1
    count_years_exist.append(years_exist)
    count_months_exist.append(months_exist)
    count_weeks_exist.append(weeks_exist)
    mean_forks_per_year.append(count_fork / years_exist)
    mean_forks_per_month.append(count_fork / months_exist)
    mean_forks_per_week.append(count_fork / weeks_exist)
    if is_steady(months, 12):
        steady_month.append(1)
    else:
        steady_month.append(0)
    if is_steady(weeks, 53):
        steady_week.append(1)
    else:
        steady_week.append(0)

df_fork_features = pd.DataFrame.from_dict({
    "snapshot_id": snapshot_ids,
    "count_forks": count_forks,
    "mean_forks_per_year": mean_forks_per_year,
    "mean_forks_per_month": mean_forks_per_month,
    "mean_forks_per_week": mean_forks_per_week,
    "count_years_with_forks": count_years_with_forks,
    "count_months_with_forks": count_months_with_forks,
    "count_years_with_forks": count_years_exist,
    "count_months_with_forks": count_months_exist,
    "count_weeks_with_fork": count_weeks_exist,
    "steady_month": steady_month,
    "steady_week": steady_week
    
}, orient='index').transpose()

df_fork_features.head(16)

# number of forks
# mean number of forks by year
# number of year with at least one fork
# number of month with at least one fork
# number of days with at least one fork
# longest streak of years with at least one fork
# longest streak of months with at least one fork


,snapshot_id,count_forks,mean_forks_per_year,mean_forks_per_month,mean_forks_per_week,count_years_with_forks,count_months_with_forks,count_weeks_with_fork,steady_month,steady_week
0,10000129.0,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,0.0,0.0
1,1000048.0,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,0.0,0.0
2,10002669.0,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,0.0,0.0
3,10002835.0,5418.0,903.000000,77.400000,20.292135,6.0,70.0,267.0,1.0,0.0
4,1000355.0,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,0.0,0.0
5,10004260.0,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,0.0,0.0
6,10004589.0,9.0,3.000000,0.290323,0.065217,3.0,31.0,138.0,0.0,0.0
7,1000482.0,5.0,1.250000,0.111111,0.032258,4.0,45.0,155.0,0.0,0.0
8,10005216.0,5.0,2.500000,0.357143,0.080645,2.0,14.0,62.0,0.0,0.0
9,10006621.0,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,0.0,0.0


In [15]:
df_fork_features.to_csv("/home/sv/data/fork_pattern.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/home/sv/data/fork_pattern.csv'